In [ ]:
import pickle
import numpy as np
from tensorflow.python.keras.models import load_model
from tensorflow.python.client import device_lib
from keras_preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
import os
import pickle
import random
import time
import numpy as np

from tensorflow.python.keras.models import load_model
from tensorflow.python.client import device_lib
from tensorflow.keras.layers import RepeatVector, TimeDistributed, Bidirectional

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.image import load_img, img_to_array

from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from google.colab import drive

curr_folder = '/content/drive'
drive.mount(curr_folder)


start_dir = curr_folder + '/MyDrive/NN/Pictures/'
end_dir = curr_folder + '/MyDrive/NN/End/'


path_tokenizer = curr_folder + '/MyDrive/NN/tokenizer.pkl'
path_train_dict = curr_folder + '/MyDrive/NN/train_dict.pkl'
path_val_dict = curr_folder + '/MyDrive/NN/val_dict.pkl'

path_features_vgg16 = curr_folder + '/MyDrive/NN/vocab.pkl'
path_features_inception = curr_folder + '/MyDrive/NN/features.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

def to_lines(data):
    all_vals = list()
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

def load_train_data(train_dict_path, tokenizer_path, features_path):
    with open (train_dict_path, 'rb') as f:
        out_train_dict = pickle.load(f)
    print('кол-во подписей .............. %d' % len(out_train_dict))

    out_train_features = load_image_features(features_path, out_train_dict)

    with open (tokenizer_path, 'rb') as f:
        out_tokenizer = pickle.load(f)
    out_vocab_size = len(out_tokenizer.word_index) + 1
    print('размер словаря ............... %d' % out_vocab_size)

    out_max_words = find_max_words(out_train_dict)
    print('длина предложения в словах ... %d' % out_max_words)

    return out_train_dict, out_tokenizer, out_vocab_size, out_max_words, out_train_features

### Наборы для валидации

In [ ]:
val_dict, tokenizer, vocab_size, max_words, val_features = load_train_data(path_val_dict, path_tokenizer, path_features_inception)
max_words = 30

кол-во подписей .............. 1562
размер словаря ............... 21391
длина предложения в словах ... 31


# Оценка модели

После того, как модель обучены, мы можем оценить качество её предсказаний в тестовом наборе данных.

Мы оценим модель, создав описания для всех изображений в тестовом наборе данных и оценив эти прогнозы с помощью стандартной функции затрат.

Во-первых, нам нужно иметь возможность сгенерировать описание изображения, используя обученную модель. Это включает в себя передачу маркера начального описания "startseq", генерацию одного слова, а затем рекурсивный вызов модели с сгенерированными словами в качестве входных данных до тех пор, пока не будет достигнут конец маркера последовательности "endseq" или не будет достигнута максимальная длина описания.

Приведённая ниже функция реализует это поведение и генерирует текстовое описание с учётом обученной модели и заданного подготовленного изображения в качестве входных данных. Эта функция вызывает другую функцию map_int_to_word, чтобы отобразить целочисленное предсказание обратно в слово.

In [ ]:
def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None

# 0 seed the generation process
# 1 iterate over the whole length of the sequence
# 2 integer encode input sequence
# 3 pad input
# 4 predict next word
# 5 convert probability to integer
# 6 map integer to word
# 7 stop if we cannot map the word
# 8 append as input for generating the next word
# 9 stop if we predict the end of the sequence
def generate_caption(model, tokenizer, image, max_words):
    in_text = 'startseq'                                    # 0

    for i in range(max_words):                              # 1
        seq = tokenizer.texts_to_sequences([in_text])[0]    # 2
        seq = pad_sequences([seq], maxlen=max_words)        # 3

        y_hat = model.predict([image,seq], verbose=0)       # 4
        y_hat = np.argmax(y_hat)                            # 5

        word = map_int_to_word(y_hat, tokenizer)            # 6
        if word is None:                                    # 7
            break

        in_text += ' ' + word                               # 8

        if word == 'endseq':                                # 9
            break

    return in_text

### BLEU

Мы сгенерируем предсказания для всех изображений в тестовом наборе данных и в наборе данных для обучения.

Приведенная ниже функция с именем evaluate_model() будет оценивать обученную модель по заданному набору подписей изображений и признаков изображений. Фактические и прогнозируемые описания собираются и оцениваются вместе с использованием оценки алгоритма BLEU, который оценивает, насколько сгенерированный текст близок к ожидаемому тексту.

BLEU используется при переводе текста для оценки переведённого текста по одному или нескольким другим переводам.

In [ ]:
def evaluate_model(model, captions, images, tokenizer, max_words):
    actual, predicted = list(), list()

    for key, captions_list in captions.items():
        references = [c.split() for c in captions_list]
        y_hat = generate_caption(model, tokenizer, images[key], max_words)

        actual.append(references)
        predicted.append(y_hat.split())

    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

### Оценка VGG16

In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.547214
BLEU-2: 0.261290
BLEU-3: 0.160986
BLEU-4: 0.094975


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.574484
BLEU-2: 0.277671
BLEU-3: 0.167548
BLEU-4: 0.090645


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.569651
BLEU-2: 0.278423
BLEU-3: 0.175996
BLEU-4: 0.097475


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.562040
BLEU-2: 0.277154
BLEU-3: 0.181927
BLEU-4: 0.110951


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.574687
BLEU-2: 0.280907
BLEU-3: 0.180219
BLEU-4: 0.099226


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.580390
BLEU-2: 0.263439
BLEU-3: 0.163909
BLEU-4: 0.072590


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.586214
BLEU-2: 0.270662
BLEU-3: 0.170193
BLEU-4: 0.076256


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.567054
BLEU-2: 0.256425
BLEU-3: 0.158422
BLEU-4: 0.068956


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.548946
BLEU-2: 0.242393
BLEU-3: 0.148712
BLEU-4: 0.065577


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.524117
BLEU-2: 0.231470
BLEU-3: 0.143271
BLEU-4: 0.061557


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.517806
BLEU-2: 0.230050
BLEU-3: 0.144490
BLEU-4: 0.061777


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.510654
BLEU-2: 0.229715
BLEU-3: 0.140786
BLEU-4: 0.059004


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.526720
BLEU-2: 0.240123
BLEU-3: 0.144624
BLEU-4: 0.057683
CPU times: user 9min 42s, sys: 12.5 s, total: 9min 55s
Wall time: 9min 19s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.523348
BLEU-2: 0.236396
BLEU-3: 0.142093
BLEU-4: 0.056965
CPU times: user 9min 43s, sys: 12.3 s, total: 9min 55s
Wall time: 9min 18s


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.502979
BLEU-2: 0.224907
BLEU-3: 0.136318
BLEU-4: 0.055936


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.415937
BLEU-2: 0.168460
BLEU-3: 0.099650
BLEU-4: 0.037336
CPU times: user 13min 51s, sys: 17.1 s, total: 14min 8s
Wall time: 13min 15s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.495484
BLEU-2: 0.219394
BLEU-3: 0.131719
BLEU-4: 0.053408
CPU times: user 9min 53s, sys: 12.2 s, total: 10min 5s
Wall time: 9min 28s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.431496
BLEU-2: 0.179422
BLEU-3: 0.106734
BLEU-4: 0.041598
CPU times: user 13min 2s, sys: 16.1 s, total: 13min 18s
Wall time: 12min 31s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.576722
BLEU-2: 0.255762
BLEU-3: 0.154356
BLEU-4: 0.068002
CPU times: user 7min 56s, sys: 10 s, total: 8min 6s
Wall time: 7min 38s


In [ ]:
path_model = curr_folder + '/MyDrive/NN/VGG1/VGG1model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.546610
BLEU-2: 0.243145
BLEU-3: 0.145509
BLEU-4: 0.061339


## Оценка VGG16-2

In [ ]:
path_model = 'D:/models/vgg16-2/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.330890
BLEU-2: 0.132260
BLEU-3: 0.089523
BLEU-4: 0.025266


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.251889
BLEU-2: 0.133708
BLEU-3: 0.100575
BLEU-4: 0.042080
Wall time: 1h 41min 25s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.333354
BLEU-2: 0.182158
BLEU-3: 0.135445
BLEU-4: 0.059651
Wall time: 1h 18min 58s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.504368
BLEU-2: 0.283720
BLEU-3: 0.207341
BLEU-4: 0.100291
Wall time: 56min 24s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.487142
BLEU-2: 0.273837
BLEU-3: 0.201340
BLEU-4: 0.096789
Wall time: 53min 13s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.503695
BLEU-2: 0.287683
BLEU-3: 0.213201
BLEU-4: 0.106811
Wall time: 53min 24s


In [ ]:
path_model = 'D:/models/vgg16-2/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.512076
BLEU-2: 0.293938
BLEU-3: 0.219297
BLEU-4: 0.110592


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.512135
BLEU-2: 0.292193
BLEU-3: 0.217427
BLEU-4: 0.111023
Wall time: 52min 39s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.511659
BLEU-2: 0.289843
BLEU-3: 0.216488
BLEU-4: 0.110830
Wall time: 50min 47s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.508568
BLEU-2: 0.288118
BLEU-3: 0.213113
BLEU-4: 0.107240
Wall time: 49min 30s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.502268
BLEU-2: 0.286105
BLEU-3: 0.213294
BLEU-4: 0.107053
Wall time: 50min 25s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.504939
BLEU-2: 0.287782
BLEU-3: 0.215184
BLEU-4: 0.108405
Wall time: 49min 49s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501813
BLEU-2: 0.284998
BLEU-3: 0.212785
BLEU-4: 0.106470
Wall time: 51min 14s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.508468
BLEU-2: 0.288323
BLEU-3: 0.215613
BLEU-4: 0.109824
Wall time: 51min 34s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501221
BLEU-2: 0.284398
BLEU-3: 0.211127
BLEU-4: 0.105456
Wall time: 53min 59s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.500942
BLEU-2: 0.282896
BLEU-3: 0.211010
BLEU-4: 0.106437
Wall time: 51min 42s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.495775
BLEU-2: 0.280361
BLEU-3: 0.210769
BLEU-4: 0.108840
Wall time: 51min 23s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.494823
BLEU-2: 0.279827
BLEU-3: 0.209732
BLEU-4: 0.107115
Wall time: 52min 35s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.482858
BLEU-2: 0.270046
BLEU-3: 0.200387
BLEU-4: 0.099557
Wall time: 51min 12s


In [ ]:
%%time
path_model = 'D:/models/vgg16-2/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.479654
BLEU-2: 0.266615
BLEU-3: 0.198103
BLEU-4: 0.098578
Wall time: 50min 57s


## Оценка Inception

In [ ]:
%%time
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.506083
BLEU-2: 0.229848
BLEU-3: 0.117963
BLEU-4: 0.056931
CPU times: user 4min 47s, sys: 6.13 s, total: 4min 53s
Wall time: 4min 38s


In [ ]:
%%time
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.578135
BLEU-2: 0.264244
BLEU-3: 0.155192
BLEU-4: 0.061154
CPU times: user 7min 33s, sys: 9.55 s, total: 7min 42s
Wall time: 7min 15s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.584309
BLEU-2: 0.269289
BLEU-3: 0.166828
BLEU-4: 0.067834
CPU times: user 7min 57s, sys: 10 s, total: 8min 7s
Wall time: 7min 37s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.578745
BLEU-2: 0.267274
BLEU-3: 0.165344
BLEU-4: 0.075693
CPU times: user 7min 52s, sys: 9.98 s, total: 8min 2s
Wall time: 7min 32s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.575544
BLEU-2: 0.267224
BLEU-3: 0.166908
BLEU-4: 0.075349
CPU times: user 7min 57s, sys: 10 s, total: 8min 7s
Wall time: 7min 37s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.571140
BLEU-2: 0.262602
BLEU-3: 0.160878
BLEU-4: 0.070366
CPU times: user 7min 55s, sys: 9.74 s, total: 8min 4s
Wall time: 7min 36s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.563990
BLEU-2: 0.259169
BLEU-3: 0.159276
BLEU-4: 0.069845
CPU times: user 8min 5s, sys: 10.6 s, total: 8min 16s
Wall time: 7min 44s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.520316
BLEU-2: 0.235528
BLEU-3: 0.141751
BLEU-4: 0.053983
CPU times: user 9min 14s, sys: 12.2 s, total: 9min 26s
Wall time: 8min 53s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.543916
BLEU-2: 0.249947
BLEU-3: 0.152320
BLEU-4: 0.062391
CPU times: user 8min 49s, sys: 11 s, total: 9min
Wall time: 8min 28s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.570103
BLEU-2: 0.261413
BLEU-3: 0.157224
BLEU-4: 0.064615
CPU times: user 8min 4s, sys: 9.96 s, total: 8min 14s
Wall time: 7min 45s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.572328
BLEU-2: 0.260745
BLEU-3: 0.154265
BLEU-4: 0.065935
CPU times: user 8min 3s, sys: 10.1 s, total: 8min 13s
Wall time: 7min 44s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.559946
BLEU-2: 0.252203
BLEU-3: 0.152400
BLEU-4: 0.066170
CPU times: user 8min 13s, sys: 10.7 s, total: 8min 24s
Wall time: 7min 55s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.572345
BLEU-2: 0.260171
BLEU-3: 0.155195
BLEU-4: 0.063577
CPU times: user 7min 59s, sys: 10.4 s, total: 8min 9s
Wall time: 7min 40s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.571155
BLEU-2: 0.259261
BLEU-3: 0.153889
BLEU-4: 0.066470
CPU times: user 8min 4s, sys: 10.2 s, total: 8min 14s
Wall time: 7min 44s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.569178
BLEU-2: 0.258399
BLEU-3: 0.154278
BLEU-4: 0.065862
CPU times: user 8min, sys: 10.3 s, total: 8min 11s
Wall time: 7min 41s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.571094
BLEU-2: 0.257536
BLEU-3: 0.151733
BLEU-4: 0.063091
CPU times: user 7min 59s, sys: 9.85 s, total: 8min 8s
Wall time: 7min 38s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.559461
BLEU-2: 0.250141
BLEU-3: 0.149292
BLEU-4: 0.062105
CPU times: user 8min 24s, sys: 10.3 s, total: 8min 34s
Wall time: 8min 3s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.535523
BLEU-2: 0.243932
BLEU-3: 0.149909
BLEU-4: 0.063928
CPU times: user 9min 5s, sys: 11.8 s, total: 9min 16s
Wall time: 8min 42s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.584450
BLEU-2: 0.272487
BLEU-3: 0.167949
BLEU-4: 0.074304
CPU times: user 8min, sys: 10.4 s, total: 8min 11s
Wall time: 7min 41s


In [ ]:
%%time 
path_model = curr_folder + '/MyDrive/NN/Inspiration1/Inspiration1-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.558483
BLEU-2: 0.255677
BLEU-3: 0.158967
BLEU-4: 0.070888
CPU times: user 8min 40s, sys: 11.1 s, total: 8min 51s
Wall time: 8min 18s


## Оценка Inception-2

In [ ]:
%%time
path_model = 'D:/models/inception-2/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.143182
BLEU-2: 0.059739
BLEU-3: 0.041356
BLEU-4: 0.009241
Wall time: 2h 10min 37s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.404583
BLEU-2: 0.192803
BLEU-3: 0.136315
BLEU-4: 0.055770
Wall time: 1h 2min


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.493209
BLEU-2: 0.259008
BLEU-3: 0.187620
BLEU-4: 0.086226
Wall time: 51min 12s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.510724
BLEU-2: 0.280957
BLEU-3: 0.207807
BLEU-4: 0.101033
Wall time: 53min 8s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.506085
BLEU-2: 0.274217
BLEU-3: 0.202119
BLEU-4: 0.098165
Wall time: 48min 9s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.513603
BLEU-2: 0.282965
BLEU-3: 0.209564
BLEU-4: 0.104223
Wall time: 51min 28s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.520624
BLEU-2: 0.293764
BLEU-3: 0.219084
BLEU-4: 0.111367
Wall time: 50min 17s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.515529
BLEU-2: 0.289828
BLEU-3: 0.215429
BLEU-4: 0.109241
Wall time: 49min 32s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.519564
BLEU-2: 0.295451
BLEU-3: 0.220886
BLEU-4: 0.113214
Wall time: 45min 9s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.521632
BLEU-2: 0.297048
BLEU-3: 0.221886
BLEU-4: 0.114537
Wall time: 48min 3s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.511789
BLEU-2: 0.286977
BLEU-3: 0.212708
BLEU-4: 0.106995
Wall time: 51min 58s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.503644
BLEU-2: 0.280418
BLEU-3: 0.205865
BLEU-4: 0.102691
Wall time: 56min 10s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.497646
BLEU-2: 0.277767
BLEU-3: 0.205065
BLEU-4: 0.101724
Wall time: 55min 5s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491665
BLEU-2: 0.274565
BLEU-3: 0.201928
BLEU-4: 0.098994
Wall time: 54min 7s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491949
BLEU-2: 0.278453
BLEU-3: 0.208454
BLEU-4: 0.104314
Wall time: 54min 45s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.497122
BLEU-2: 0.279421
BLEU-3: 0.208215
BLEU-4: 0.105135
Wall time: 54min 17s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491076
BLEU-2: 0.274560
BLEU-3: 0.204636
BLEU-4: 0.104065
Wall time: 53min 2s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.486741
BLEU-2: 0.273443
BLEU-3: 0.203743
BLEU-4: 0.102320
Wall time: 53min 43s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.490434
BLEU-2: 0.273504
BLEU-3: 0.202311
BLEU-4: 0.101990
Wall time: 52min 3s


In [ ]:
%%time
path_model = 'D:/models/inception-2/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.490287
BLEU-2: 0.274459
BLEU-3: 0.204949
BLEU-4: 0.105633
Wall time: 54min 12s
